# 图像金字塔与轮廓检测
### 图像金字塔主要有两个：高斯金字塔与拉普拉斯金字塔

In [1]:
#导入相关库与包
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 高斯金字塔

In [2]:
#定义一个图片显示函数
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
#高斯金字塔应用
bi = cv2.imread("labixiaoxin.jfif")
cv_show("bi",bi)
print(bi.shape)

(422, 470, 3)


In [5]:
#高斯金字塔向上采样
bi_up = cv2.pyrUp(bi)
cv_show("bi_up",bi_up)
print(bi_up.shape)

(844, 940, 3)


In [8]:
#高斯金字塔向下采样
bi_down = cv2.pyrDown(bi)
cv_show("bi_down",bi_down)
print(bi_down.shape)

(211, 235, 3)


In [16]:
#将bi_up继续向上采样
bi_up2 = cv2.pyrUp(bi_up)
cv_show("bi_up2",bi_up2)
print(bi_up2.shape)
cv2.imwrite("bi_up.jpg",bi_up2)     #可以看出，保存的图片比原图片大了不少，约16倍吧

(1688, 1880, 3)


True

In [11]:
#将bi_up2向下采样，使其变回bi_up
bi_up3 = cv2.pyrDown(bi_up2)
cv_show("bi_up3",bi_up3)
print(bi_up3.shape)

(844, 940, 3)


In [18]:
#原图像与up_down图像对比
bi_up_down = cv2.pyrDown(bi_up)
cv_show("up_down",np.hstack((bi,bi_up_down)))
#可以发现，原图像会比较清晰

# 拉普拉斯金字塔

In [21]:
img = cv2.imread("flowers.jfif")
img_down = cv2.pyrDown(img)
down_up = cv2.pyrUp(img_down)
Laplascian = img - down_up
cv_show("Laplascian",Laplascian)
cv2.imwrite("Laplace_flowers.jpg",Laplascian)

True

In [20]:
cv_show("img",img)
#经过Laplace金字塔（感觉是轮廓信息了），与原图的区别还是比较明显的。

# 图像轮廓

In [24]:
#为了更高的准确率，使用二值图像
pi_img = cv2.imread("pi.jfif")
gray = cv2.cvtColor(pi_img,cv2.COLOR_BGR2GRAY)      #把图像转化为灰度图
ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)  #得到了二值的结果
cv_show("thresh",thresh)

In [25]:
binary,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

### 绘制轮廓

In [31]:
#传入绘制图像，轮廓，轮廓索引，颜色模式，线条厚度
#注意需要copy,要不原图会变
draw_img = pi_img.copy()
res = cv2.drawContours(draw_img,contours,-1,(0,0,255),2)        #-1默认全部轮廓
cv_show("res",res)

In [30]:
#注意区别
res = cv2.drawContours(draw_img,contours,-1,(0,0,255),2)    #0，0，255 （BGR）想用什么颜色绘制就改变就好。
cv_show("res",res)

In [27]:
draw_img = pi_img.copy()
res = cv2.drawContours(draw_img,contours,0,(0,0,255),2)
cv_show("res",res)

### 轮廓特征

In [32]:
cat = contours[0]

In [33]:
#计算面积，需要把每个轮廓拿出来算才可以
cv2.contourArea(cat)

421201.0

In [34]:
#周长，True表示轮廓是闭合的
cv2.arcLength(cat,True)

2596.0

### 轮廓的近似

In [36]:
img = cv2.imread("dance.jfif")

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
binary,contours,hieraachy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
cat = contours[0]

draw_img = img.copy()
res = cv2.drawContours(draw_img,[cat],-1,(0,0,255),2)
cv_show("res",res)

In [42]:
epsilon = 0.1*cv2.arcLength(cat,True)
approx = cv2.approxPolyDP(cat,epsilon,True)    #第一参数：指定的轮廓；第二参数：阈值（作比较）；

draw_img = img.copy()
res = res = cv2.drawContours(draw_img,[approx],-1,(0,0,255),2)
cv_show("res",res)

### 边界矩形

In [47]:
img = cv2.imread("ciecle.jfif")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
binary,contours,hieraachy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
cat = contours[2]

x,y,w,h = cv2.boundingRect(cat)
img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
cv_show("img",img)

In [48]:
area = cv2.contourArea(cat)
x,y,w,h = cv2.boundingRect(cat)
rect_area = w*h
extent = float(area)/rect_area
print("轮廓面积与边界矩形比",extent)

轮廓面积与边界矩形比 0.34285714285714286


# 总结